In [1]:
# Runs Grid Cell Percentile and FHP Metrics 
import sys
sys.path.append('/n/home02/pbb/scripts/halo-metadata-server/')
from Cloud_Class import Cloud, calccover
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import time
import rasterio as rio
import laspy
import time
import json
import concurrent.futures
import pickle
from pathlib import Path
import geopandas as gpd

### Define Inputs

In [6]:
# # # DEFINE USER INPUTS

# LAS Inputs
lasdir = '/n/davies_lab/Users/pbb/StructuralComplexity_Tyler/data/in/UHURU_Plots_las'

lasinputs = glob.glob(f'{lasdir}/**/*.las', recursive=True)

# Define Grid

# 25 cm voxels
xysize = 0.25
verticalres = 0.25

# Output directory 
outdir = '/n/davies_lab/Users/pbb/StructuralComplexity_Tyler/data/out/UHURU_100Plots_metrics'


# Outdirectory for rasters
outdir_rast = '/n/davies_lab/Users/pbb/StructuralComplexity_Tyler/data/out/UHURU_100Plots_rasters'

# Shapefile of Plots (for iterively setting boundaries of the grid)
# NOTE: This is the dissolved file without a buffer
# each feature is a plot polygon, marked by Site and Block number
shpf = Path('/n/home02/pbb/scripts/halo-metadata-server/StructuralComplexity_Tyler/data/in/UHURU_Polygons/UHURU_100mPlots_Dissolved.shp')
shpdf = gpd.read_file(shpf)

minh=0
maxh=10
groundthres=0.03

# # # END USER INPUTS

### Define Function for Calculating Cover in Parallel

In [3]:
# Wrapper function for using parallel processing and calccover function 
# Notice that is calls PlotCloud as the first argument (an input specific to this script)
# It has to be this way in order to use concurrent futures parallel processing below.
def calccover_parallel(index):

    # make a True/False array 
    # for all points within the current grid cell
    idx_bool = PlotCloud.grid_dict['idx_points'] == PlotCloud.grid_dict['idx_cells'][index]
    
    # Calculate Cover - treating anything [<3 cm as a ground hit]
    cover = calccover(points=PlotCloud.las.points[idx_bool],
                      step=PlotCloud.vsize,
                      groundthres=groundthreshold,
                      heightcol=PlotCloud.heightcol,
                      hmax=maxh)

    # Calculate Percentile Metrics of Height
    perc_dict= {0:[],
                25:[],
                50:[],
                75:[],
                98:[],
                100:[],
                'mean':[],
                'std':[]}
    
    # Get Heights for given cell
    heights = PlotCloud.las.points[PlotCloud.heightcol][idx_bool]
    
    # Make sure heights only includes points > ground threshold
    heights = heights[heights >= groundthres]
    
    # If there are any heights left
    if heights.size > 0:
        
        perc_dict[0].append(np.quantile(heights, [0]).flat[0])
        perc_dict[25].append(np.quantile(heights, [0.25]).flat[0])
        perc_dict[50].append(np.quantile(heights, [0.5]).flat[0])
        perc_dict[75].append(np.quantile(heights, [0.75]).flat[0])
        perc_dict[98].append(np.quantile(heights, [0.98]).flat[0])
        perc_dict[100].append(np.quantile(heights, [1.0]).flat[0])
        perc_dict['mean'].append(np.mean(heights).flat[0])
        perc_dict['std'].append(np.std(heights).flat[0])
    
    # else, height stats are 0
    else:
        perc_dict[0].append(0)
        perc_dict[25].append(0)
        perc_dict[50].append(0)
        perc_dict[75].append(0)
        perc_dict[98].append(0)
        perc_dict[100].append(0)
        perc_dict['mean'].append(0)
        perc_dict['std'].append(0)

    # Get Intensity for the given cell
    # intensity = PlotCloud.las.points['Intensity'][idx_bool]
    
#     # Fill dictionary for intensity metrics
#     int_dict= {0:[],
#                25:[],
#                50:[],
#                75:[],
#                98:[],
#                100:[],
#                'mean':[],
#                'std':[]}
    
#     int_dict[0].append(np.quantile(intensity, [0]).flat[0])
#     int_dict[25].append(np.quantile(intensity, [0.25]).flat[0])
#     int_dict[50].append(np.quantile(intensity, [0.5]).flat[0])
#     int_dict[75].append(np.quantile(intensity, [0.75]).flat[0])
#     int_dict[98].append(np.quantile(intensity, [0.98]).flat[0])
#     int_dict[100].append(np.quantile(intensity, [1.0]).flat[0])
#     int_dict['mean'].append(np.mean(intensity).flat[0])
#     int_dict['std'].append(np.std(intensity).flat[0])
     
    if calculatecover:
        # Return cover dict, percentile dict, and height list (for quick recalculation of anything later)
        return cover, perc_dict, heights
    else:
        return perc_dict, heights

In [7]:
# filter las inputs (if you only need to run a subset of the files)
# lasinputs = lasinputs[2:]
# lasinputs

['/n/davies_lab/Users/pbb/StructuralComplexity_Tyler/data/in/UHURU_Plots_las/UHURUSouth/UHURUSouth_South_3.las',
 '/n/davies_lab/Users/pbb/StructuralComplexity_Tyler/data/in/UHURU_Plots_las/UHURUCentral/UHURUCentral_Central_2.las',
 '/n/davies_lab/Users/pbb/StructuralComplexity_Tyler/data/in/UHURU_Plots_las/UHURUCentral/UHURUCentral_Central_3.las',
 '/n/davies_lab/Users/pbb/StructuralComplexity_Tyler/data/in/UHURU_Plots_las/UHURUCentral/UHURUCentral_Central_1.las',
 '/n/davies_lab/Users/pbb/StructuralComplexity_Tyler/data/in/UHURU_Plots_las/UHURUNorth/UHURUNorth_North_3.las',
 '/n/davies_lab/Users/pbb/StructuralComplexity_Tyler/data/in/UHURU_Plots_las/UHURUNorth/UHURUNorth_North_2.las',
 '/n/davies_lab/Users/pbb/StructuralComplexity_Tyler/data/in/UHURU_Plots_las/UHURUNorth/UHURUNorth_North_1.las']

### Compute Metrics and Save to Pickle - Working Version 7/29/22

In [ ]:
for lasf in lasinputs:

    ### STEP 1: Load in Cloud 
    startproj = time.time()

    # Make cloud object and set grid size and vertical res for FHP metrics
    PlotCloud = Cloud(lasf=lasf,
                      gridsize=xysize,
                      vsize=verticalres,
                      heightcol='HeightAboveGround')

    # get project string for saving below
    projstr = PlotCloud.lasf.split('/')[-1].split('.')[0]

    end = time.time()
    
    print(f'Loaded {projstr} cloud, time elapsed: {end - startproj}\n')
    
    ### STEP 2: Make the grid
    start = time.time()
    
    # *** ADDITIONAL STEP SPECIFIC TO UHURU ***
    # Set the boundaries of the grid using the corresponding feature in the Plots Shapefile
    
    # Find the site and the block from the las file name
    Site = lasf.split('/')[-1].split('_')[1]
    Block = int(lasf.split('/')[-1].split('_')[-1].split('.')[0])
    
    # Match site and block
    feat_gs = shpdf.loc[ ((shpdf.Site==Site) & (shpdf.Block == Block))]
    
    # Make a grid using the above boundaries
    PlotCloud.makegrid(xmin=float(feat_gs.geometry.bounds.minx),
                       ymin=float(feat_gs.geometry.bounds.miny),
                       xmax=float(feat_gs.geometry.bounds.maxx),
                       ymax=float(feat_gs.geometry.bounds.maxy))

    end = time.time()
    
    print(f'Grid created, time elapsed: {end - start}\n')
    
    ### STEP 3: Compute Cover, FHP, and Percentiles Metrics Over the Cloud's Grid
    start = time.time()

    # initialize cover dictionary for output 
    PlotCloud.cover_dict = {}
    PlotCloud.perc_dict = {}
    PlotCloud.height_dict = {}

    # set the cell indices to loop over in parallel
    indices = PlotCloud.grid_dict['idx_cells']

    # Use concurrent futures to compute cover over each cell in parallel
    with concurrent.futures.ProcessPoolExecutor(max_workers=None) as executor:
        for cph, x, y in zip(executor.map(calccover_parallel, indices),
                                 PlotCloud.grid_dict['x_cells'],
                                 PlotCloud.grid_dict['y_cells']):

            # Stick the cover, perc, and heights inside the metrics dictionary
            # with x and y location as tuple keys
            PlotCloud.cover_dict[(x, y)] = cph[0]
            PlotCloud.perc_dict[(x, y)] = cph[1]
            PlotCloud.height_dict[(x, y)] = np.round(cph[2], decimals=3)

    end = time.time()
    
    print(f'Metrics computed, time elapsed: {end - start}\n')
    
    # Save outputs as pickles
    # "Can't open a pickle you don't know" - can be malicious pickles, be wary.
    with open(f'{outdir}/{projstr}_{xysize}mgrid_covermetrics.obj', 'wb') as of:
        pickle.dump(PlotCloud.cover_dict, of, protocol=pickle.HIGHEST_PROTOCOL)

    with open(f'{outdir}/{projstr}_{xysize}mgrid_percmetrics.obj', 'wb') as of:
        pickle.dump(PlotCloud.perc_dict, of, protocol=pickle.HIGHEST_PROTOCOL)

    with open(f'{outdir}/{projstr}_{xysize}mgrid_heights.obj', 'wb') as of:
        pickle.dump(PlotCloud.height_dict, of, protocol=pickle.HIGHEST_PROTOCOL)
    
    # DONE
    endproj = time.time()
    projtime = endproj - startproj
    
    numcells = len(PlotCloud.grid_dict['idx_cells'])
    print(f'Done with processing {projstr}.\n')
    print(f'{PlotCloud.las.header.point_count} points gridded into {numcells} {xysize}m pixels in {projtime} seconds!\n')

Loaded UHURUSouth_South_3 cloud, time elapsed: 3.32749342918396

Grid created, time elapsed: 72.79837608337402



/n/home02/pbb/scripts/halo-metadata-server/Cloud_Class.py:221: RuntimeWarning: divide by zero encountered in log
  FHPD1 = -np.log(1-coverD1byH) / np.diff(np.array(heightbins))
/n/home02/pbb/scripts/halo-metadata-server/Cloud_Class.py:221: RuntimeWarning: divide by zero encountered in log
  FHPD1 = -np.log(1-coverD1byH) / np.diff(np.array(heightbins))
/n/home02/pbb/scripts/halo-metadata-server/Cloud_Class.py:222: RuntimeWarning: divide by zero encountered in log
  FHPD2 = -np.log(1-coverD2byH) / np.diff(np.array(heightbins))
/n/home02/pbb/scripts/halo-metadata-server/Cloud_Class.py:222: RuntimeWarning: divide by zero encountered in log
  FHPD2 = -np.log(1-coverD2byH) / np.diff(np.array(heightbins))
/n/home02/pbb/scripts/halo-metadata-server/Cloud_Class.py:221: RuntimeWarning: divide by zero encountered in log
  FHPD1 = -np.log(1-coverD1byH) / np.diff(np.array(heightbins))
/n/home02/pbb/scripts/halo-metadata-server/Cloud_Class.py:222: RuntimeWarning: divide by zero encountered in log
  

Metrics computed, time elapsed: 7227.256954908371

Done with processing UHURUSouth_South_3.

55056501 points gridded into 1043292 0.25m pixels in 7414.070305585861 seconds!

Loaded UHURUCentral_Central_2 cloud, time elapsed: 9.176736116409302

Grid created, time elapsed: 118.78048515319824



/n/home02/pbb/scripts/halo-metadata-server/Cloud_Class.py:221: RuntimeWarning: divide by zero encountered in log
  FHPD1 = -np.log(1-coverD1byH) / np.diff(np.array(heightbins))
/n/home02/pbb/scripts/halo-metadata-server/Cloud_Class.py:222: RuntimeWarning: divide by zero encountered in log
  FHPD2 = -np.log(1-coverD2byH) / np.diff(np.array(heightbins))
/n/home02/pbb/scripts/halo-metadata-server/Cloud_Class.py:221: RuntimeWarning: divide by zero encountered in log
  FHPD1 = -np.log(1-coverD1byH) / np.diff(np.array(heightbins))
/n/home02/pbb/scripts/halo-metadata-server/Cloud_Class.py:221: RuntimeWarning: divide by zero encountered in log
  FHPD1 = -np.log(1-coverD1byH) / np.diff(np.array(heightbins))
/n/home02/pbb/scripts/halo-metadata-server/Cloud_Class.py:221: RuntimeWarning: divide by zero encountered in log
  FHPD1 = -np.log(1-coverD1byH) / np.diff(np.array(heightbins))
/n/home02/pbb/scripts/halo-metadata-server/Cloud_Class.py:221: RuntimeWarning: divide by zero encountered in log
  

Metrics computed, time elapsed: 15667.52705693245

Done with processing UHURUCentral_Central_2.

79747102 points gridded into 1528741 0.25m pixels in 15949.258023023605 seconds!

Loaded UHURUCentral_Central_3 cloud, time elapsed: 11.858437776565552

Grid created, time elapsed: 68.16583704948425



/n/home02/pbb/scripts/halo-metadata-server/Cloud_Class.py:221: RuntimeWarning: divide by zero encountered in log
  FHPD1 = -np.log(1-coverD1byH) / np.diff(np.array(heightbins))
/n/home02/pbb/scripts/halo-metadata-server/Cloud_Class.py:222: RuntimeWarning: divide by zero encountered in log
  FHPD2 = -np.log(1-coverD2byH) / np.diff(np.array(heightbins))
/n/home02/pbb/scripts/halo-metadata-server/Cloud_Class.py:221: RuntimeWarning: divide by zero encountered in log
  FHPD1 = -np.log(1-coverD1byH) / np.diff(np.array(heightbins))
/n/home02/pbb/scripts/halo-metadata-server/Cloud_Class.py:221: RuntimeWarning: divide by zero encountered in log
  FHPD1 = -np.log(1-coverD1byH) / np.diff(np.array(heightbins))
/n/home02/pbb/scripts/halo-metadata-server/Cloud_Class.py:221: RuntimeWarning: divide by zero encountered in log
  FHPD1 = -np.log(1-coverD1byH) / np.diff(np.array(heightbins))
/n/home02/pbb/scripts/halo-metadata-server/Cloud_Class.py:221: RuntimeWarning: divide by zero encountered in log
  

Metrics computed, time elapsed: 6547.60298538208

Done with processing UHURUCentral_Central_3.

47594818 points gridded into 1101143 0.25m pixels in 6746.591368198395 seconds!

Loaded UHURUCentral_Central_1 cloud, time elapsed: 13.178991079330444

Grid created, time elapsed: 127.62927842140198



/n/home02/pbb/scripts/halo-metadata-server/Cloud_Class.py:221: RuntimeWarning: divide by zero encountered in log
  FHPD1 = -np.log(1-coverD1byH) / np.diff(np.array(heightbins))
/n/home02/pbb/scripts/halo-metadata-server/Cloud_Class.py:222: RuntimeWarning: divide by zero encountered in log
  FHPD2 = -np.log(1-coverD2byH) / np.diff(np.array(heightbins))
/n/home02/pbb/scripts/halo-metadata-server/Cloud_Class.py:221: RuntimeWarning: divide by zero encountered in log
  FHPD1 = -np.log(1-coverD1byH) / np.diff(np.array(heightbins))
/n/home02/pbb/scripts/halo-metadata-server/Cloud_Class.py:222: RuntimeWarning: divide by zero encountered in log
  FHPD2 = -np.log(1-coverD2byH) / np.diff(np.array(heightbins))
/n/home02/pbb/scripts/halo-metadata-server/Cloud_Class.py:221: RuntimeWarning: divide by zero encountered in log
  FHPD1 = -np.log(1-coverD1byH) / np.diff(np.array(heightbins))
/n/home02/pbb/scripts/halo-metadata-server/Cloud_Class.py:221: RuntimeWarning: divide by zero encountered in log
  

### Compute Metrics (v2 - timed out below, don't use!)

In [1]:
for lasf in [lasinputs[0]]:

    ### STEP 1: Load in Cloud 
    startproj = time.time()

    # Make cloud object and set grid size and vertical res for FHP metrics
    PlotCloud = Cloud(lasf=lasf,
                      gridsize=xysize,
                      vsize=verticalres,
                      heightcol='HeightAboveGround')

    # get project string for saving below
    projstr = PlotCloud.lasf.split('/')[-1].split('.')[0]

    end = time.time()
    
    print(f'Loaded {projstr} cloud, time elapsed: {end - startproj}\n')
    
    ### STEP 2: Make the grid
    start = time.time()
    
    # *** ADDITIONAL STEP SPECIFIC TO UHURU ***
    # Set the boundaries of the grid using the corresponding feature in the Plots Shapefile
    
    # Find the site and the block from the las file name
    Site = lasf.split('/')[-1].split('_')[1]
    Block = int(lasf.split('/')[-1].split('_')[-1].split('.')[0])
    
    # Match site and block
    feat_gs = shpdf.loc[ ((shpdf.Site==Site) & (shpdf.Block == Block))]
    
    # Set bounds of grid to fill
    xmin=float(feat_gs.geometry.bounds.minx)
    ymin=float(feat_gs.geometry.bounds.miny)
    xmax=float(feat_gs.geometry.bounds.maxx)
    ymax=float(feat_gs.geometry.bounds.maxy)
    
    # Make a grid using the above boundaries (lower left coords of cells here)
    PlotCloud.makegrid(xmin=xmin,
                       ymin=ymin,
                       xmax=xmax,
                       ymax=ymax)

    end = time.time()
    
    print(f'Grid created, time elapsed: {end - start}\n')
    
    ### STEP 3: Compute Cover, FHP, and Percentiles Metrics Over the Cloud's Grid
    start = time.time()

    # initialize cover, percentile, and intensity dictionaries for output 
    PlotCloud.cover_dict = {}
    PlotCloud.perc_dict = {}
    PlotCloud.height_dict = {}
    
    # initialize 2D and 3D arrays for geotif and netcdf outputs
    # Make the coordinates of the grid
    x_grid = np.arange(xmin, xmax, step=xysize)
    y_grid = np.arange(ymin, ymax, step=xysize)
    # also use height 
    z_grid = np.arange(0, 10, step=0.25)

    # Mesh the grid into 2 matrices of x and y coordinates
    x_mesh, y_mesh = np.meshgrid(x_grid, y_grid)
    
    # Also, grid of 3D matrices of xyz coords
    # x_mesh3, y_mesh3, z_mesh3 = np.meshgrid(x_grid, y_grid, z_grid)

    # Make empty 2D metric arrays, filled with nans
    
    # Make metric arrays of equal size to the x and y mesh, filled with nans
    
    # Cover metrics
    cov0_array = np.full(x_mesh.shape, np.nan)
    cov0p5_array = np.full(x_mesh.shape, np.nan)
    cov1p5_array = np.full(x_mesh.shape, np.nan)

    # Number of Pulses
    npulses_array = np.full(x_mesh.shape, np.nan)

    # PAI
    pai0_array = np.full(x_mesh.shape, np.nan)
    pai0p5_array = np.full(x_mesh.shape, np.nan)
    pai1p5_array = np.full(x_mesh.shape, np.nan)

    # Percentiles
    perc0_array = np.full(x_mesh.shape, np.nan)
    perc25_array = np.full(x_mesh.shape, np.nan)
    perc50_array = np.full(x_mesh.shape, np.nan)
    perc75_array = np.full(x_mesh.shape, np.nan)
    perc98_array = np.full(x_mesh.shape, np.nan)
    perc100_array = np.full(x_mesh.shape, np.nan)
    meanh_array = np.full(x_mesh.shape, np.nan)
    stdh_array = np.full(x_mesh.shape, np.nan)
    
    # set the cell indices to loop over in parallel
    indices = PlotCloud.grid_dict['idx_cells']

    # Use concurrent futures to compute cover over each cell in parallel
    # https://docs.python.org/3/library/concurrent.futures.html#concurrent.futures.ProcessPoolExecutor
    with concurrent.futures.ProcessPoolExecutor(max_workers=None) as executor:
        for cph, x, y in zip(executor.map(calccover_parallel, indices),
                                 PlotCloud.grid_dict['x_cells'],
                                 PlotCloud.grid_dict['y_cells']):
            
            iterstart = time.time()
            
            key = (x, y)

            # Stick the cover, perc, and heights inside the metrics dictionary
            # with x and y location as tuple keys
            PlotCloud.cover_dict[key] = cph[0]
            PlotCloud.perc_dict[key] = cph[1]
            PlotCloud.height_dict[key] = np.round(cph[2], decimals=3)
            
            # Fill 2D arrays
            
            # Get the index location to fill the metric array, using xmesh and ymesh matrices
            idx = ((x_mesh==x) & (y_mesh==y))
            
            # For each key in the perc dict
            # for key in cph[1].keys():
                
            # Fill the 2D arrays with the location values

            # percentiles
            perc0_array = np.where(idx, PlotCloud.perc_dict[key][0][0], perc0_array)
            perc25_array = np.where(idx, PlotCloud.perc_dict[key][25][0], perc25_array)
            perc50_array = np.where(idx, PlotCloud.perc_dict[key][50][0], perc50_array)
            perc75_array = np.where(idx, PlotCloud.perc_dict[key][75][0], perc75_array)
            perc98_array = np.where(idx, PlotCloud.perc_dict[key][98][0], perc98_array)
            perc100_array = np.where(idx, PlotCloud.perc_dict[key][100][0], perc100_array)
            meanh_array = np.where(idx, PlotCloud.perc_dict[key]['mean'][0], meanh_array)
            stdh_array = np.where(idx, PlotCloud.perc_dict[key]['std'][0], std_array)

            # Fill the arrays with the corresponding value
            cov0_array = np.where(idx, PlotCloud.cover_dict[key]['CoverD2'][1], cov0_array)
            cov0p5_array = np.where(idx, PlotCloud.cover_dict[key]['CoverD2'][2], cov0p5_array)
            cov1p5_array = np.where(idx, PlotCloud.cover_dict[key]['CoverD2'][5], cov1p5_array)

            try:
                npulses_array = np.where(idx, cover[key]['Npulses'][0], npulses_array)
            except:
                npulses_array = np.where(idx, 0, npulses_array)

            pai0_array = np.where(idx, cover[key]['FHPD2'][1], pai0_array)
            pai0p5_array = np.where(idx, cover[key]['FHPD2'][2], pai0p5_array)
            pai1p5_array = np.where(idx, cover[key]['FHPD2'][5], pai1p5_array)

            iterend = time.time()

            print(f'Iteration done in {iterend-iterstart}')
                
    end = time.time()
    
    print(f'Metrics computed, time elapsed: {end - start}\n')
    
    # Save outputs as pickles
    # "Can't open a pickle you don't know" - There can be malicious pickles, be wary.
    with open(f'{outdir}/{projstr}_{xysize}mgrid_covermetrics.obj', 'wb') as of:
        pickle.dump(PlotCloud.cover_dict, of, protocol=pickle.HIGHEST_PROTOCOL)

    with open(f'{outdir}/{projstr}_{xysize}mgrid_percmetrics.obj', 'wb') as of:
        pickle.dump(PlotCloud.perc_dict, of, protocol=pickle.HIGHEST_PROTOCOL)

    with open(f'{outdir}/{projstr}_{xysize}mgrid_heights.obj', 'wb') as of:
        pickle.dump(PlotCloud.height_dict, of, protocol=pickle.HIGHEST_PROTOCOL)
        
    # Print
    print(f'Pickles for {Site} {Block} saved. \n')
        
    # Save outputs as geotifs
    # Write to raster
    for ar, l in zip([cov0_array, cov0p5_array, cov1p5_array, npulses_array,
                      pai0_array, pai0p5_array, pai1p5_array, perc0_array,
                      perc25_array,perc50_array, perc75_array, perc98_array,
                      perc100_array, mean_array, std_array],
                     ['Cover 0m', 'Cover 0.5m', 'Cover 1.5m', 
                     'PAI Above 0m', 'PAI Above 0.5m', 'PAI Above 1.5m', 'Npulses',
                     '0th Percentile Height [m]', '25th Percentile Height [m]',
                     '50th Percentile Height [m]', '75th Percentile Height [m]',
                     '98th Percentile Height [m]', '100th Percentile Height [m]',
                     'Mean Height of All Points [m]', 'StD Height of All Points [m]']):

        out_xr = xr.DataArray(data=ar, coords=[y_grid, x_grid], dims=["y", "x"])
        out_xr.rio.write_crs("epsg:32637", inplace=True)
        label = l.replace(' ', '').replace('.', 'p').replace('[m]', '')
        out_xr.rio.to_raster(f'{outdir_rast}/{label}_{Site}{str(Block)}.tif')

    # Save 3D outputs as netcdfs
    
    # DONE
    endproj = time.time()
    projtime = endproj - startproj
    
    numcells = len(PlotCloud.grid_dict['idx_cells'])
    print(f'Done with processing {projstr}.\n')
    print(f'{PlotCloud.las.header.point_count} points gridded into {numcells} {xysize}m pixels in {projtime} seconds!\n')### Compute Metrics

NameError: name 'lasinputs' is not defined

In [18]:
PlotCloud.perc_dict.keys()

dict_keys([(265479.5045180492, 31198.849579518905)])

In [20]:
cph[-1]

array([], dtype=float32)

In [16]:
x, y

(265479.5045180492, 31198.849579518905)

In [ ]:
cph[0]

## Testing

In [12]:
cph[1].keys()

dict_keys([0, 25, 50, 75, 98, 100, 'mean', 'std'])